In [ ]:
from functions_miryam import *
from scipy.linalg import svd
from scipy.linalg import eig
from numpy import *
import numpy as np
import pandas as pd
from cluster_louvain import *

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.utils.extmath import randomized_svd
import operator

import matplotlib.cm as cm
import networkx as nx

data = pd.read_csv("snn.csv", encoding= 'unicode_escape')
l=data.shape
print(l)
Adj=data.to_numpy()
print(np.shape(Adj))
G = nx.from_numpy_matrix(Adj)

In [ ]:
#Generating the true labels

n=4743 #dataset1

#Size of each clusters
setlistn=[1022,924,260,955,839,743] #CD4, 8, 14, 19, 34, 56

ideni=np.zeros((7))
for i in range(6):
    ideni[i+1]=ideni[i]+setlistn[i]


iden=np.zeros((n))

for i in range(6):
    for j in range(round(ideni[i]),round(ideni[i+1])):
        iden[j]=i

In [ ]:
#Initialize and Loading
current=0
n=4743

k=round(0.02*n)
print("n=",n,"k=",k)

#scomp6.npy is the sorted pairs of vertices according to PCA compression Ratio
scomp=np.load('scomp6.npy')

sol=np.zeros(n)
for i in range(n):
    sol[i]=-1

t=0
for i in range(10000):
    if(scomp[i][1]==scomp[i][2]):
        t=t+1

print("Number of intra-cluster edges of original graph in 10,000",t)
n1=n
current=0

#what's the scomp?



In [ ]:
print(np.shape(scomp),(n*(n-1)//2))

In [ ]:
#New-approach-only-plural-and-merge

firstiden=[]

ll1=n*(n-1)//2
ty=np.shape(scomp)
ll2=ty[0]

t1=0
pcn=0
per=0.03
lp=n1

stopper=per*1000*2//3
print("Stopper for this case=",stopper)

while(lp>stopper):

    print("Creating Virtual Cluster",current)
    
    # sort nodes according to inner product with largest eigenvector
    n1,nadj,nadj1,cur,rcur,gU,tdeg=makegraph(sol,iden,n,per,scomp)
    sproj=pluralset(nadj1,n1,pcn,gU)
    sorted_node=[]
    for j in range(st):
        sorted_node.append(sproj[j][1])

    # plot inner product values
    sorted_innerproduct=np.zeros(n1)
    for i in range(n1):
        sorted_innerproduct[i]=sproj[i][0]
    printplot(sorted_innerproduct,n1)

    # ad-hoc
    if(n1//2>2000):
        st=2000
    else:
        st=n1//2
    
    # binary search
    # BUG?
    l=10
    r=st    
    while(l<r):
        mid=(l+r)//2
        avg1=mean_degree(sorted_node[0:mid],n1,nadj)

        #changing from //2 to //1.5
        if(avg1>mid//2):
            l=mid+1
        else:
            r=mid-1

    print("For this round, clique size=",mid) 
    avg1=mean_degree(sorted_node[0:mid//2],n1,nadj)
    avg2=mean_degree(sorted_node[mid+100],n1,nadj)
    print("Density for first",mid//2,"is",avg1,"and for first",mid+100,"is",avg2)
    
    cutoff=mid
    
    track1=[]
    for i in range(cutoff):
        t11=sproj[i][1]
        track1.append(t11)
        
    pcount=clusterid(track1,cur)
    print("Plural set first calc:",pcount)
    
    rset=filterset(track1,nadj,n1)
    degcount=clusterid(rset,cur)
        
    print("Plural set after filtration:",degcount)
    lp=len(rset)

    pos=majorityid(rset,cur)
    firstiden.append(pos)
    print("Firstiden for this cluster is:",)

    current=recovernew(rset,n1,sol,current,cur)

print("We have finished with",current,"many clusters and ",n1,"many unassigned cells") 

In [ ]:
#Storing confident sets separately. 
confset,remain=clusterify(sol,current,n)

In [ ]:
print(len(confset))

In [ ]:
#We start embedding using Louvain from here. 
print(len(confset))
cc=0

initial_cluster=np.zeros((n))

for i in range(current):
    for j in range(len(confset[i])):
        cc=cc+1
        t=round(j)
        t1=round(confset[i][t])
        initial_cluster[t1]=i

for i in range(len(remain)):
    cc=cc+1
    t1==round(remain[i])
    initial_cluster[t1]=current+i

print("The numeber of vertices marked=",cc,"number of clusters=",current+i)






In [ ]:
data = pd.read_csv("snn.csv", encoding= 'unicode_escape')
l=data.shape
print(l)
Adj=data.to_numpy()
print(np.shape(Adj))
G = nx.from_numpy_matrix(Adj)

In [ ]:
# compute the best leveled cluster
l_cluster = best_leveled_cluster(G, initial_cluster,resolution=0.5)
loutput=[]
for i in range(n):
    loutput.append(l_cluster[i])


print(max(loutput))

In [ ]:
print(len(loutput))

louvainiden=seuratcluster(loutput)
cluster(louvainiden,iden)


In [ ]:
#Pre-merging step
#We also have firsiden

firstidenstore=firstiden

sol1=sol
current0=current
print(firstidenstore,firstiden)


In [ ]:
firstiden=firstidenstore

In [ ]:
for i in range(current):
    if((firstiden[i]==4 and len(confset[i])==31) or (firstiden[i]==2 and len(confset[i])==14)):
        
        for j in range(len(confset[i])):
            print(confset[i][j]+1,",",end='')

In [ ]:
#Makeset

ty=np.shape(scomp)
ll=ty[0]
#ll=(n*(n-1))//2
k=round(0.015*n)


print(k)

extractor=np.zeros((n,k))

for i in range(n):
    for j in range(k):
        extractor[i][j]=-1

counter=np.zeros((n))


for i in range(ll):
    x=round(scomp[i][3])
    y=round(scomp[i][4])

    if(counter[x]<k-1):
        xx=round(counter[x])
        extractor[x][xx]=y
        counter[x]=counter[x]+1

    if(counter[y]<k-1):
        yy=round(counter[y])
        extractor[y][yy]=x
        counter[y]=counter[y]+1

In [ ]:
#Pairwise symmetric based merging
current=current0
firstiden=[]
for i in range(current):
    firstiden.append(firstidenstore[i])

firstiden1=firstiden
print(current,firstiden,firstidenstore,len(firstiden),len(firstiden1),len(firstidenstore))

stopp=0
sol1=sol
current1=current
preiden,remain=clusterify(sol,current,n)
print(len(preiden))

zscore=[]

cnum=9

print("current=",current,"cc=",cnum)

for l in range(current-cnum):

    score=np.zeros((current1,current1,3))
    score1=np.zeros((current1,current1,3))

   
    for i in range(current1):
        for j in range(current1):
            co=0
            for i1 in range(len(preiden[i])):
                t1=round(preiden[i][i1])
                temp=set(extractor[t1,:])
                temp1=set(preiden[j])
                temp2=temp.intersection(temp1)

                co=co+len(temp2)


            co=co/len(preiden[i])
            score[i][j][0]=co
            score[i][j][1]=firstiden1[i]
            score[i][j][2]=firstiden1[j]
                


    for i in range(current1):
        for j in range(current1):
            score1[i][j][0]=score[i][j][0]*score[j][i][0]
            score1[i][j][1]= score[i][j][1]
            score1[i][j][2]= score[i][j][2]

    ss=current1*current1
    mergable=np.zeros((ss,5))

    t=0
    for i in range(current1):
        for j in range(current1):
                if(i!=j):
                    mergable[t][0]=score1[i][j][0]
                    mergable[t][1]=score1[i][j][1]
                    mergable[t][2]=score1[i][j][2]
                    mergable[t][3]=i 
                    mergable[t][4]=j
                
                t=t+1

    smerge=sorted(mergable, key=operator.itemgetter(0), reverse=True)

    print("Merge candidate",smerge[0])

    zscore.append(smerge[0][0])

    current1=current1-1
    m1=round(smerge[0][3])
    m2=round(smerge[0][4])

    print("The merge is between clusters",m1,m2,"With identities",smerge[0][1],smerge[0][2],"of size",len(preiden[m1]),len(preiden[m2]))

    #print("print some more values")
    #for tt in range(current1):
    #    print(smerge[tt])

    new1=preiden[m1]+preiden[m2]
    pos1=firstiden[m1]


    index=[m1,m2]
    preiden = [e for i, e in enumerate(preiden) if i not in index]
    firstiden = [e for i, e in enumerate(firstiden) if i not in index]

    #preiden.pop(m1,m2)
    #preiden.pop(m2)
    #firstiden.pop(m1,m2)
    #firstiden.pop(m2)

    preiden1=[]
    for i in range(current1):
         preiden1.append([])

    
    firstiden1=[]

    preiden1[0]=new1
    firstiden1.append(pos1)
    

    print(len(firstiden1),len(firstiden))
      
    for i in range(1,current1):
        preiden1[i]=preiden[i-1]
        firstiden1.append(firstiden[i-1])

    

    preiden=preiden1
    firstiden=firstiden1

    print("Going for the next merge")


print("Finsihed merging with",len(preiden),"many confident clusters")


In [ ]:
y=[]
y1=[]
for i in range(len(zscore)):
    y1.append(current-i)
    y.append(i)




fig = plt.figure(figsize=(16, 6), dpi=80)
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.set_xticks(y)
ax.set_xticklabels(y1)

plt.xlabel("Number of clusters at the point of merge")
plt.ylabel("Z_max value for the merge")
plt.plot(zscore)
plt.savefig("zmax.png")
plt.show()    


In [ ]:
#storing confident clusters

l2=len(preiden)

confcluster=[]
for i in range(l2):
    confcluster.append(preiden[i])



In [ ]:
print(len(preiden))

In [ ]:
print(len(preiden))
cc=0

initial_cluster=np.zeros((n))

for i in range(len(preiden)):
    for j in range(len(preiden[i])):
        cc=cc+1
        t=round(j)
        t1=round(preiden[i][t])
        initial_cluster[t1]=i

for i in range(len(remain)):
    cc=cc+1
    t1==round(remain[i])
    initial_cluster[t1]=len(preiden)+i

print("The numeber of vertices marked=",cc,"number of clusters=",len(preiden)+i)


In [ ]:
l_cluster = best_leveled_cluster(G, initial_cluster,resolution=2)
loutput=[]
for i in range(n):
    loutput.append(l_cluster[i])


print(max(loutput))

In [ ]:
print(len(loutput))

louvainiden=seuratcluster(loutput)
cluster(louvainiden,iden)

In [ ]:
#Symmetric Merging.


print(cnum,cln)

for i in range(cnum):
    t=len(preiden[i])
    degcount=np.zeros((cln))

    for j in range(t):
        t=round(preiden[i][j])
        t1=round(iden[t])
        degcount[t1]=degcount[t1]+1

    print(degcount)

In [ ]:
cluster(confcluster,iden)

In [ ]:
#Majority vote
print(k)

ch=1
lold=0
while(ch!=0):

    unmatched=np.zeros((n))
    for i in range(n):
        unmatched[i]=-1


    preiden1=[]
    for i in range(cnum):
            preiden1.append([])

    for i in range(cnum):
        for j in range(len(preiden[i])):
            t=preiden[i][j]
            unmatched[t]=0

    remain=[]
    for i in range(n):
        if(unmatched[i]==-1):
            remain.append(i)

    print("Remaining vertices=",len(remain))






    for i in range(cnum):
        temp1=set(preiden[i])
        for j in range(len(remain)):
            t1=round(remain[j])
            temp2=set(extractor[t1,:])

            temp3=temp1.intersection(temp2)

            l1=len(temp3)

            if(l1>k//2):
                preiden1[i].append(t1)

    for i in range(cnum):
        preiden[i]=preiden[i]+preiden1[i] 
        

    for i in range(cnum):
        t=len(preiden[i])
        degcount=np.zeros((cln))

        for j in range(t):
            t=round(preiden[i][j])
            t1=round(iden[t])
            degcount[t1]=degcount[t1]+1

        print(degcount.astype(int))

    
    if(len(remain)==lold):
        ch=0

    lold=len(remain)





In [ ]:
#storing first round of clusters

l2=len(preiden)

finalcluster1=[]
for i in range(l2):
    finalcluster1.append(preiden[i])

In [ ]:
cluster(finalcluster1,iden)

In [ ]:
print("Number of remaining vertices",len(remain))

In [ ]:
print("hello test")